In [2]:
import numpy as np
np.random.seed(1337)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils
import matplotlib.pyplot as plt
import pickle
import plotting

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
image_dim = 512

# raw_data = pickle.load(open('data-lausanne-{}.p'.format(image_dim), 'rb'))
raw_data = pickle.load(open('images-lausanne-zoom-17.p', 'rb'))
X = np.array(raw_data['sat'])
y = np.array(raw_data['style'])
# y = y.reshape(y.shape[0], image_dim**2)

# style image to fraction of image that are trees
old_y = y
t = []
for yi in old_y:
    ws = np.sum(yi)
    ts = yi.size
    t.append(ws / ts)
old_y = y
y = np.array(t)

print(X.shape)
print(y.shape)

(2650, 512, 512, 3)
(2650,)


In [4]:
perm = np.random.permutation(len(X))
X = X[perm]
y = y[perm]
test_set_size = 0.15
split_index = int(len(X) * (1 - test_set_size))
X_train = X[:split_index]
y_train = y[:split_index]
X_test = X[split_index:]
y_test = y[split_index:]

In [5]:
num_train, height, width, depth = X_train.shape
num_test = X_test.shape[0]

X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_train /= np.max(X_train)
X_test /= np.max(X_test)

Y_train = y_train
Y_test = y_test
# Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
# Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels
print(Y_train.shape)

(2252,)


In [6]:
model = Sequential()
model.add(Conv2D(32, (2, 2), activation='relu', input_shape=(image_dim, image_dim, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

In [7]:
model.compile(loss='mean_squared_error', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 511, 511, 32)      416       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 509, 509, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 254, 254, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 254, 254, 64)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4129024)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               528515200 
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [9]:
hist = model.fit(X_train,
                     Y_train,
                     batch_size=32,
                     epochs = 50,
                     validation_data=(X_test, Y_test))
plotting.plot_history(hist, "32 tiles, 50 epochs, 32 batch size")

Train on 2252 samples, validate on 398 samples
Epoch 1/50
  64/2252 [..............................] - ETA: 2:00:27 - loss: 203224.8467 - acc: 0.1250

KeyboardInterrupt: 